In [ ]:
!pip -q install pydantic-ai
!pip -q install nest_asyncio
!pip -q install devtools

### **Restart the notebook**

In [ ]:
import os
from google.colab import userdata
from IPython.display import display, Markdown

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_AI_STUDIO')
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from pydantic_ai import Agent, ModelRetry

agent = Agent(
    'gemini-1.5-flash',
    system_prompt='Be very concise, reply with one sentence only.',
    retries=3
)

result = agent.run_sync('Where does "hello world" come from?')
print(result.data)

In [ ]:
agent.model = 'gemini-1.5-pro'

@agent.system_prompt
async def get_system_prompt(self) -> str:
    return "Give a long one paragraph answer and make it dense"

In [ ]:

result = agent.run_sync('Where does "hello world" come from?')
Markdown(result.data)

## Basic structured output

In [ ]:
import os
from typing import cast

from pydantic import BaseModel

from pydantic_ai import Agent
from pydantic_ai.models import KnownModelName

class MyModel(BaseModel):
    city: str
    country: str
    reason: str
    famous_person_from_city: str


model = 'openai:gpt-4o'
print(f'Using model: {model}')

agent = Agent(model, result_type=MyModel)

result = agent.run_sync('The windy city in the US of A.')
print(result.data)

In [ ]:
print(result.cost())

In [ ]:
result = agent.run_sync('The Merlion city.')
print(result.data)

In [ ]:
result = agent.run_sync('The cold city in the south')
print(result.data)

In [ ]:
result.data.famous_person_from_city

In [ ]:
agent

## Chat APP

In [ ]:
from pydantic_ai import Agent
from pprint import pprint

In [ ]:
agent = Agent('openai:gpt-4o-mini', system_prompt='Be a helpful assistant.')

In [ ]:
result = agent.run_sync('Tell me a joke.')
print(result.data)

In [ ]:
# all messages from the run
pprint(result.all_messages())

In [ ]:
type(result.all_messages())

In [ ]:
pprint(result.new_messages())

In [ ]:
result2 = agent.run_sync('Explain it please?', message_history=result.new_messages())
print(result2.data)

In [ ]:
pprint(result2.all_messages())

In [ ]:
pprint(result2.new_messages())

In [ ]:
result3 = agent.run_sync('Where does the joke come from?', message_history=result2.all_messages())
print(result3.data)

In [ ]:
pprint(result3.all_messages())

In [ ]:
print(result3.all_messages_json())

## Chatting LLMs

In [ ]:
from pydantic_ai import Agent

agent = Agent('openai:gpt-4o-mini', system_prompt='Be a helpful assistant.')

result1 = agent.run_sync('Tell me a joke.')
print(result1.data)

result2 = agent.run_sync('Explain?', model='gemini-1.5-pro', message_history=result1.new_messages())
print(result2.data)

In [ ]:
pprint(result2.all_messages())

## Weather

In [ ]:
from __future__ import annotations as _annotations

import asyncio
import os
from dataclasses import dataclass
from typing import Any

from devtools import debug
from httpx import AsyncClient

from pydantic_ai import Agent, ModelRetry, RunContext

In [ ]:
@dataclass
class Deps:
    client: AsyncClient
    weather_api_key: str | None
    geo_api_key: str | None


In [ ]:
weather_agent = Agent(
    'openai:gpt-4o',
    system_prompt='Be concise, reply with one sentence.',
    deps_type=Deps,
    retries=2,
)

@weather_agent.tool
async def get_lat_lng(
    ctx: RunContext[Deps], location_description: str
) -> dict[str, float]:
    """Get the latitude and longitude of a location.

    Args:
        ctx: The context.
        location_description: A description of a location.
    """
    if ctx.deps.geo_api_key is None:
        # if no API key is provided, return a dummy response (London)
        return {'lat': 51.1, 'lng': -0.1}

    params = {
        'q': location_description,
        'api_key': ctx.deps.geo_api_key,
    }
    with logfire.span('calling geocode API', params=params) as span:
        r = await ctx.deps.client.get('https://geocode.maps.co/search', params=params)
        r.raise_for_status()
        data = r.json()
        span.set_attribute('response', data)

    if data:
        return {'lat': data[0]['lat'], 'lng': data[0]['lon']}
    else:
        raise ModelRetry('Could not find the location')


@weather_agent.tool
async def get_weather(ctx: RunContext[Deps], lat: float, lng: float) -> dict[str, Any]:
    """Get the weather at a location.

    Args:
        ctx: The context.
        lat: Latitude of the location.
        lng: Longitude of the location.
    """
    if ctx.deps.weather_api_key is None:
        # if no API key is provided, return a dummy response
        return {'temperature': '21 °C', 'description': 'Sunny'}

    params = {
        'apikey': ctx.deps.weather_api_key,
        'location': f'{lat},{lng}',
        'units': 'metric',
    }
    with logfire.span('calling weather API', params=params) as span:
        r = await ctx.deps.client.get(
            'https://api.tomorrow.io/v4/weather/realtime', params=params
        )
        r.raise_for_status()
        data = r.json()
        span.set_attribute('response', data)

    values = data['data']['values']
    # https://docs.tomorrow.io/reference/data-layers-weather-codes
    code_lookup = {
        1000: 'Clear, Sunny',
        1100: 'Mostly Clear',
        1101: 'Partly Cloudy',
        1102: 'Mostly Cloudy',
        1001: 'Cloudy',
        2000: 'Fog',
        2100: 'Light Fog',
        4000: 'Drizzle',
        4001: 'Rain',
        4200: 'Light Rain',
        4201: 'Heavy Rain',
        5000: 'Snow',
        5001: 'Flurries',
        5100: 'Light Snow',
        5101: 'Heavy Snow',
        6000: 'Freezing Drizzle',
        6001: 'Freezing Rain',
        6200: 'Light Freezing Rain',
        6201: 'Heavy Freezing Rain',
        7000: 'Ice Pellets',
        7101: 'Heavy Ice Pellets',
        7102: 'Light Ice Pellets',
        8000: 'Thunderstorm',
    }
    return {
        'temperature': f'{values["temperatureApparent"]:0.0f}°C',
        'description': code_lookup.get(values['weatherCode'], 'Unknown'),
    }

In [ ]:
async def main():
    async with AsyncClient() as client:
        # create a free API key at https://www.tomorrow.io/weather-api/
        weather_api_key = os.getenv('WEATHER_API_KEY')
        # create a free API key at https://geocode.maps.co/
        geo_api_key = os.getenv('GEO_API_KEY')
        deps = Deps(
            client=client, weather_api_key=weather_api_key, geo_api_key=geo_api_key
        )
        result = await weather_agent.run(
            'What is the weather like in London and in Singapore?', deps=deps
        )
        debug(result)
        print('Response:', result.data)

In [ ]:
asyncio.run(main())

## Streaming MarkDown

In [ ]:
import asyncio
import os

from rich.console import Console, ConsoleOptions, RenderResult
from rich.live import Live
from rich.markdown import CodeBlock, Markdown
from rich.syntax import Syntax
from rich.text import Text

from pydantic_ai import Agent
from pydantic_ai.models import KnownModelName


agent = Agent()

# models to try, and the appropriate env var
models: list[tuple[KnownModelName, str]] = [
    ('gemini-1.5-flash', 'GEMINI_API_KEY'),
    ('openai:gpt-4o-mini', 'OPENAI_API_KEY'),
    ('groq:llama-3.1-70b-versatile', 'GROQ_API_KEY')
]


async def main():
    prettier_code_blocks()
    console = Console()
    prompt = 'Show me a short example of using Pydantic.'
    console.log(f'Asking: {prompt}...', style='cyan')
    for model, env_var in models:
        if env_var in os.environ:
            console.log(f'Using model: {model}')
            with Live('', console=console, vertical_overflow='visible') as live:
                async with agent.run_stream(prompt, model=model) as result:
                    async for message in result.stream():
                        live.update(Markdown(message))
            console.log(result.cost())
        else:
            console.log(f'{model} requires {env_var} to be set.')


def prettier_code_blocks():
    """Make rich code blocks prettier and easier to copy.

    From https://github.com/samuelcolvin/aicli/blob/v0.8.0/samuelcolvin_aicli.py#L22
    """

    class SimpleCodeBlock(CodeBlock):
        def __rich_console__(
            self, console: Console, options: ConsoleOptions
        ) -> RenderResult:
            code = str(self.text).rstrip()
            yield Text(self.lexer_name, style='dim')
            yield Syntax(
                code,
                self.lexer_name,
                theme=self.theme,
                background_color='black',
                word_wrap=True,
            )
            yield Text(f'/{self.lexer_name}', style='dim')

    Markdown.elements['fence'] = SimpleCodeBlock


if __name__ == '__main__':
    asyncio.run(main())